# Different rank profiles on Vespa

This notebook provides examples for single-document passage retrieval for bm25, dense, hybrid-dense and hybrid-sparse retrieval in Vespa. It'll eventually be integrated directly into RAG, but this will also be useful as a playground for debugging.

Note the following environment variables are needed to run this `VESPA_URL`, `VESPA_CERT_LOCATION`, `VESPA_KEY_LOCATION`.

In [76]:
import os

from vespa.application import Vespa
from vespa.io import VespaQueryResponse
from vespa.exceptions import VespaError
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [7]:
app = Vespa(
    url=os.environ["VESPA_URL"],
    cert=os.environ["VESPA_CERT_LOCATION"],
    key=os.environ["VESPA_KEY_LOCATION"],
)

app.get_application_status()

Using mTLS (key,cert) Authentication against endpoint https://a6a329f3.acc2de3b.z.vespa-app.cloud/ApplicationStatus


<Response [200]>

## hybrid request

In [92]:
models = dict()

models["bge_small"] = SentenceTransformer("BAAI/bge-small-en-v1.5")
models["bge_base"] = SentenceTransformer("BAAI/bge-base-en-v1.5")
models["distilbert_dot_v5"] = SentenceTransformer("sentence-transformers/msmarco-distilbert-dot-v5")
models["distilbert_base_tas_b"] = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-tas-b")

/Users/kalyan/Library/Caches/pypoetry/virtualenvs/rag-labs-4_BbfzMU-py3.9/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [1]:

rank_profiles = [f"dense_{model_name}" for model_name in models.keys()] + [f"hybrid_{model_name}" for model_name in models.keys()] + ["bm25", "splade"]

def make_request(
    query: str, 
    document_id: str,
    hits: int = 20, 
    rank_profile: str = "hybrid_bge_small",
    hybrid_bm25_weight: float = 1,
) -> dict:
    
    if rank_profile not in rank_profiles:
        raise ValueError(f"Invalid rank_profile. Must be one of {rank_profiles}")
    
    if rank_profile not in {"splade", "bm25"}:
        model_name = "_".join(rank_profile.split("_")[1:])
        embedding_model = models[model_name]
        query_embedding = embedding_model.encode(query, convert_to_numpy=True, show_progress_bar=False).tolist()
        query_body = {
            "input.query(query_embedding)": query_embedding,
        }
        yql=f"select text_block_id, text_block, text_block_window from sources document_passage where (userQuery() or ({{targetHits:1000}}nearestNeighbor(text_embedding_{model_name},query_embedding))) and (document_import_id in ('{document_id}'))"
        
        if "hybrid" in rank_profile:
            query_body["input.query(bm25_weight)"] = hybrid_bm25_weight
        
    else:
        query_body = None
        yql = f"select text_block_id, text_block from sources document_passage where userQuery() and (document_import_id in ('{document_id}'))"
        
        
    with app.syncio() as session:
        response: VespaQueryResponse = session.query(
            yql=yql,
            hits=hits,
            query=query,
            ranking=rank_profile,
            body=query_body
        )

    return response.json

make_request(
    "how does the document define resilient development?",
    "CCLW.executive.8737.1422",
    hits=2,
    rank_profile="hybrid_bge_small",
)

NameError: name 'models' is not defined

In [104]:
def compare_requests(
    query: str,
    document_id: str,
    hits: int = 10,
    hybrid_bm25_weight: float = 0.02,
):
    rank_profiles = [
        "bm25",
        "splade",
        "dense_bge_small", 
        "dense_bge_base", 
        "dense_distilbert_dot_v5", 
        "dense_distilbert_base_tas_b", 
        "hybrid_bge_small", 
        "hybrid_bge_base", 
        "hybrid_distilbert_dot_v5", 
        "hybrid_distilbert_base_tas_b", 
    ]
    
    results = dict()
    
    for profile in tqdm(rank_profiles):
        try:
            results[profile] = make_request(
                query=query,
                document_id=document_id,
                hits=hits,
                rank_profile=profile,
                hybrid_bm25_weight=hybrid_bm25_weight,
            )
        except VespaError as e:
            results[profile] = e
            print(f"Error with rank profile {profile}: {e}")
            
    
    for profile in rank_profiles:
        print(f"Results for rank profile *{profile}*")
        for idx in range(hits):
            # print(f"Hit {idx}")
            if isinstance(results[profile], VespaError):
                print(f"{profile}: {results[profile]}")
            else:
                print(f"- {results[profile]['root']['children'][idx]['fields']['text_block']}")
        print("\n")
        
    return results
            
results = compare_requests(
    query="how does the document define resilient development?",
    document_id="CCLW.executive.8737.1422",
)

  0%|          | 0/10 [00:00<?, ?it/s]

Results for rank profile *bm25*
- Low Emission Climate Resilient Development
- define
- Towards Low Carbon Climate Resilient Development
- BOX 1: Kenya's low carbon climate resilient development pathway
- A low carbon climate resilient development pathway for Kenya emphasises:
- Box 1: Kenya's low carbon climate resilient development pathway 5
- Kenya's National Climate Change Response Strategy was the first national policy document on climate change. It sought to advance the integration of climate change adaptation and mitigation into all government planning, budgeting, and development objectives.
- climate resilient
- knowledge management refers to the organisation and sharing of climate change knowledge, while climate change-related capacity development is defined by the UNFCCC as "enhancing the capacity and ability of countries to take effective climate change action."138 The priority actions on capacity development, presented in Table 10, emphasise the establishment of engendered
